In [1]:
import numpy as np
%matplotlib qt5
import matplotlib.pyplot as pl
import os
import pickle


In [2]:
%load_ext autoreload

from sep_decoder_2 import sep_dynamics_2
from quantum_decoder import quantum_dynamics_2
from biased_decoder import biased_dynamics
from circuit_numpy_utils import get_circuit, get_diffusion_constants
from circuit_generation_multiple_ancilla import get_circuit_parameters

%autoreload 2

/Users/utkarshagrawal/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/Users/utkarshagrawal/Documents/Postdoc/U_1_exp/Weak measurements
test


In [3]:
prop_cycle = pl.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color'][:]


In [4]:
indices = list(range(6))
rng = np.random.default_rng(1)
rng.shuffle(indices)
print(indices)
rng.shuffle(indices)
print(indices)

[4, 0, 2, 1, 5, 3]
[2, 1, 3, 5, 4, 0]


In [5]:

def load_data(filename='nothing_fixed'):

    with open(filename,'rb') as f:
        data = pickle.load(f)

    return data

def get_ancilla_data(data,L_list,temp_quan=1,p_selected=[]):
    ent = {}
    err = {}
    var = {}
    var_err = {}
    for i,L in enumerate(L_list[:]):
        ent[L] = {}
        err[L] = {}
        var[L] = {} 
        var_err[L] = {}
        p_list = list(sorted(data[L].keys()))
        # p_list = [0.412,0.518,0.624,0.73,0.836]
        p_data = []
        for p in p_list:
            if len(p_selected)>0:
                if p not in p_selected:
                    continue
            p_data.append(p)
            if temp_quan == 0:
                # print(p,L)
                # print(data[L][p])
                N = len(data[L][p][L//2])
                if L!=14: N=N
                tempQ = list(data[L][p][L//2])[:N]
                N = len(data[L][p][L//2-1])
                if L!=14: N=N
                tempQ2 = list(data[L][p][L//2-1])[:N]
            else:
                N = len(data[L][p][L//2])
                if L!=14: N=N
                tempQ = [data[L][p][L//2][i][-1] for i in range(N)]
                N = len(data[L][p][L//2-1])
                if L!=14: N=N
                tempQ2 = [data[L][p][L//2-1][i][-1] for i in range(N)]
            

            temp = np.array(tempQ + tempQ2)
            N = len(temp)
            print("L=",L,"p=",p,"data_size:",len(tempQ),len(tempQ2))

            entropy = np.array([(-x*np.log(x) - (1-x)*np.log(1-x)) if 0<x<1 else 0 for x in temp])
            ydata = entropy
            avg = np.average(np.array(ydata))
            mu_2 = np.average((np.array(ydata)-avg)**2)
            mu_4 = np.average((np.array(ydata)-avg)**4)
            ent[L][p] = avg
            err[L][p] = (np.std(np.array(ydata))/(N-1)**0.5)
            var[L][p] = mu_2
            var_err[L][p] = np.sqrt((mu_4-(N-3)*mu_2**2/(N-1))/N)

    return ent,err,var,var_err


def get_suc_prob_data(data,L_list,temp_quan=1,p_selected=[]):
    ent = {}
    err = {}
    var = {}
    var_err = {}
    for i,L in enumerate(L_list[:]):
        ent[L] = {}
        err[L] = {}
        var[L] = {} 
        var_err[L] = {}
        p_list = list(sorted(data[L].keys()))
        # p_list = [0.412,0.518,0.624,0.73,0.836]
        p_data = []
        for p in p_list:
            if len(p_selected)>0:
                if p not in p_selected:
                    continue
            p_data.append(p)
            if temp_quan == 0:
                # print(p,L)
                # print(data[L][p])
                N = len(data[L][p][L//2])
                if L!=14: N=N
                tempQ = list(data[L][p][L//2])[:N]
                N = len(data[L][p][L//2-1])
                if L!=14: N=N
                tempQ2 = list(data[L][p][L//2-1])[:N]
            else:
                N = len(data[L][p][L//2])
                if L!=14: N=N
                tempQ = [data[L][p][L//2][i][-1] for i in range(N)]
                N = len(data[L][p][L//2-1])
                if L!=14: N=N
                tempQ2 = [data[L][p][L//2-1][i][-1] for i in range(N)]
            

            temp = np.array(tempQ + tempQ2)
            N = len(temp)
            print("L=",L,"p=",p,"data_size:",len(tempQ),len(tempQ2))

            entropy = np.array([(-x*np.log(x) - (1-x)*np.log(1-x)) if 0<x<1 else 0 for x in temp])
            ydata = temp
            avg = np.average(np.array(ydata))
            mu_2 = np.average((np.array(ydata)-avg)**2)
            mu_4 = np.average((np.array(ydata)-avg)**4)
            ent[L][p] = avg
            err[L][p] = (np.std(np.array(ydata))/(N-1)**0.5)
            var[L][p] = mu_2
            var_err[L][p] = np.sqrt((mu_4-(N-3)*mu_2**2/(N-1))/N)

    return ent,err,var,var_err

def get_binder_data(data,L_list,temp_quan=1,p_selected=[]):
    binder = {}
    for i,L in enumerate(L_list[:]):
        binder[L] = {}
        p_list = list(sorted(data[L].keys()))
        # p_list = [0.412,0.518,0.624,0.73,0.836]
        p_data = []
        for p in p_list:
            if len(p_selected)>0:
                if p not in p_selected:
                    continue
            p_data.append(p)
            if temp_quan == 0:
                # print(p,L)
                # print(data[L][p])
                N = len(data[L][p][L//2])
                if L!=14: N=N
                tempQ = list(data[L][p][L//2])[:N]
                N = len(data[L][p][L//2-1])
                if L!=14: N=N
                tempQ2 = list(data[L][p][L//2-1])[:N]
            else:
                N = len(data[L][p][L//2])
                if L!=14: N=N
                tempQ = [data[L][p][L//2][i][-1] for i in range(N)]
                N = len(data[L][p][L//2-1])
                if L!=14: N=N
                tempQ2 = [data[L][p][L//2-1][i][-1] for i in range(N)]
            

            temp = np.array(tempQ + tempQ2)
            N = len(temp)
            print("L=",L,"p=",p,"data_size:",len(tempQ),len(tempQ2))
            entropy = np.array([(-x*np.log(x) - (1-x)*np.log(1-x)) if 0<x<1 else 0 for x in temp])

            ydata = entropy
            avg = np.average(np.array(ydata))
            mu_2 = np.average((np.array(ydata)-avg)**2)
            mu_4 = np.average((np.array(ydata)-avg)**4)
            binder[L][p] = (1-mu_4/(3*mu_2**2))


    return binder



def get_acc_data(data,L_list,temp_quan=1,p_selected=[]):
    ent = {}
    err = {}
    var = {}
    var_err = {}
    for i,L in enumerate(L_list[:]):
        ent[L] = {}
        err[L] = {}
        var[L] = {} 
        var_err[L] = {}
        p_list = list(sorted(data[L].keys()))
        # p_list = [0.412,0.518,0.624,0.73,0.836]
        p_data = []
        for p in p_list:
            if len(p_selected)>0:
                if p not in p_selected:
                    continue
            p_data.append(p)
            if temp_quan == 0:
                # print(p,L)
                # print(data[L][p])
                N = len(data[L][p][L//2])
                if L!=14: N=N
                tempQ = list(data[L][p][L//2])[:N]
                N = len(data[L][p][L//2-1])
                if L!=14: N=N
                tempQ2 = list(data[L][p][L//2-1])[:N]
            else:
                N = len(data[L][p][L//2])
                if L!=14: N=N
                tempQ = [data[L][p][L//2][i][-1] for i in range(N)]
                N = len(data[L][p][L//2-1])
                if L!=14: N=N
                tempQ2 = [data[L][p][L//2-1][i][-1] for i in range(N)]
            

            temp = np.array(tempQ + tempQ2)
            N = len(temp)
            print("L=",L,"p=",p,"data_size:",len(tempQ),len(tempQ2))

            entropy = np.array([(-x*np.log(x) - (1-x)*np.log(1-x)) if 0<x<1 else 0 for x in temp])
            ydata = temp>0.5
            avg = np.average(np.array(ydata))
            mu_2 = np.average((np.array(ydata)-avg)**2)
            mu_4 = np.average((np.array(ydata)-avg)**4)
            ent[L][p] = avg
            err[L][p] = (np.std(np.array(ydata))/(N-1)**0.5)
            var[L][p] = mu_2
            var_err[L][p] = np.sqrt((mu_4-(N-3)*mu_2**2/(N-1))/N)

    return ent,err,var,var_err


## obselete function below
def plot_success_prob(data,L_list,charge_fac = 0,N_samples=-1,marker='o',ls='-',post_label = '',color_list = [],ms=6,temp_quan=1,p_selected=[],alpha=1,do_label=True):
    ent,err,_,_ = get_suc_prob_data(data,L_list,temp_quan=temp_quan,p_selected=p_selected)
    for i,L in enumerate(L_list):
        p_data = sorted(list(ent[L].keys()))
        ydata = [ent[L][p] for p in p_data]
        errdata = [err[L][p] for p in p_data]
        if not do_label:
            label = ''
        else:
            label='L='+str(L)+post_label

        if not color_list:
            pl.errorbar(np.array(p_data)/(np.pi/2),ydata,yerr=errdata,ms=ms,ls=ls,marker=marker,label=label,alpha=alpha)
        else:
            pl.errorbar(np.array(p_data)/(np.pi/2),ydata,yerr=errdata,ms=ms,color=color_list[i],ls=ls,marker=marker,label=label,alpha=alpha)

    pl.xlabel(r'$\gamma$',fontsize=16)
    # pl.ylabel(r'$\langle p \rangle_{suc}$',fontsize=16)
    pl.ylabel(r'$P_{\rm corr}$',fontsize=16)
    # pl.ylabel(r'Accuracy',fontsize=16)
    pl.legend(fontsize=11)
    pl.tight_layout()

In [6]:

#obselete functions below
def plot_success_binder(data,L_list,charge_fac = 0,N_samples=-1,marker='o',ls='-',post_label = '',do_label=True,color_list = [],ms=6,temp_quan=1,p_selected=[],alpha=1):
    binder = {}
    err = {}
    for i,L in enumerate(L_list[:]):
        binder[L] = []
        err[L] = []
        p_list = list(sorted(data[L].keys()))
        # p_list = [0.412,0.518,0.624,0.73,0.836]
        p_data = []
        for p in p_list:
            if len(p_selected)>0:
                if p not in p_selected:
                    continue
            p_data.append(p)
            if temp_quan == 0:
                # print(p,L)
                # print(data[L][p])
                N = len(data[L][p][L//2])
                if L!=14: N=N
                tempQ = list(data[L][p][L//2])[:N]
                N = len(data[L][p][L//2-1])
                if L!=14: N=N
                tempQ2 = list(data[L][p][L//2-1])[:N]
            else:
                N = len(data[L][p][L//2])
                if L!=14: N=N
                tempQ = [data[L][p][L//2][i][-1] for i in range(N)]
                N = len(data[L][p][L//2-1])
                if L!=14: N=N
                tempQ2 = [data[L][p][L//2-1][i][-1] for i in range(N)]
            

            temp = tempQ + tempQ2
            print("L=",L,"p=",p,"data_size:",len(tempQ),len(tempQ2))
            # ent[L].append(np.sum(np.array(tempQ+tempQ2)>0.5)/len(tempQ+tempQ2))
            # err[L].append(0)
            ent_list = np.array([(-x*np.log(x) - (1-x)*np.log(1-x)) if 0<x<1 else 0 for x in tempQ2+tempQ])

            ent_list = np.array(temp)
            
            mu_1 = np.average(ent_list)
            mu_4 = np.average((ent_list-mu_1)**4)
            e_4 = np.std(ent_list**4)/len(tempQ+tempQ2)
            mu_2 = np.average((ent_list-mu_1)**2)
            e_2 = 2*mu_2*np.std(ent_list**2)/len(tempQ+tempQ2)
            # binder[L].append(1-mu_4/(3*mu_2**2))
            # err[L].append((e_4/mu_2 + mu_4*e_2/(mu_2**2))*0)

            N = len(temp)
            binder[L].append(mu_2)
            err[L].append(np.sqrt((mu_4-(N-3)*mu_2**2/(N-1))/N))
        if not do_label:
            label = ''
        else:
            label='L='+str(L)+post_label
        if not color_list:
            pl.errorbar(np.array(p_data)/(np.pi/2),binder[L],yerr=err[L],ms=ms,ls=ls,marker=marker,label=label,alpha=alpha)
        else:
            pl.errorbar(np.array(p_data)/(np.pi/2),binder[L],yerr=err[L],ms=ms,color=color_list[i],ls=ls,marker=marker,label=label,alpha=alpha)
    pl.xlabel(r'$\gamma$',fontsize=16)
    pl.ylabel(r'Variance${}_{suc}$',fontsize=16)
    pl.legend(fontsize=10)
    pl.tight_layout()

def plot_acc_binder(data,L_list,N_samples=-1,marker='-o'):
    binder = {}
    err = {}
    for L in L_list[:]:
        binder[L] = []
        err[L] = []
        p_list = np.array(list(data[L].keys()))
        for p in p_list:
            tempQ = list(data[L][p][L//2])[:N_samples]
            tempQ2 = list(data[L][p][L//2-1])[:N_samples]
            print("L=",L,"p=",p,"data_size:",len(tempQ),len(tempQ2))
            # ent[L].append(np.sum(np.array(tempQ+tempQ2)>0.5)/len(tempQ+tempQ2))
            # err[L].append(0)
            ent_list = np.array([x for x in tempQ2+tempQ])
            mu_1 = np.average(ent_list)
            mu_4 = np.average((ent_list-mu_1)**4)
            e_4 = np.std(ent_list**4)/len(tempQ+tempQ2)
            mu_2 = np.average((ent_list-mu_1)**2)
            e_2 = 2*mu_2*np.std(ent_list**2)/len(tempQ+tempQ2)
            binder[L].append(1-mu_4/(3*mu_2**2))
            err[L].append((e_4/mu_2 + mu_4*e_2/(mu_2**2)))

        pl.errorbar(p_list,binder[L],yerr=err[L],ls='-',marker=marker,label='L='+str(L))

    pl.xlabel(r'$\gamma$',fontsize=16)
    pl.ylabel(r'Binder${}_{acc}$',fontsize=16)
    pl.legend(fontsize=16)
    pl.tight_layout()

In [6]:
L=10
Q=5
p=0.651
depth=L
seed=1
U_list = get_circuit(L,depth,'Special',seed,5)
D_list = get_diffusion_constants(circuit_layer=U_list)

filedir = 'data/qiskit_data/measurement_data_all_qubits_special'
filename = filedir +'/L='+str(L)+'_depth='+str(depth)+'_Q='+str(Q)+'_p=' + str(p)+ '_seed='+str(seed)+'.imdat'
with open(filename,'rb') as f:
    data_raw,_,_ = pickle.load(f)

/Users/utkarshagrawal/anaconda3/lib/python3.9/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/utkarshagrawal/anaconda3/lib/python3.9/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


In [8]:
result_list = []
for data in data_raw[10:11]:
    result = sep_dynamics_2(data[0],Q,p,t_scr=5,neel_state=True)
    result_list.extend([result]*data[1])
# file = 'data/biased_sep_data/seed=1_all_qubits_special_scrambled'
# data=load_data(file)


In [16]:
(result_list[0][1]).shape

(2, 2, 2, 2, 2, 2, 2, 2, 2, 2)

In [17]:
def get_RNN_hardware_data_suc_prob(L,gamma_arr_exp):
    if L==6 :num_RNN_units = 32
    else: num_RNN_units = 128
    dropout = 0.1
    RNN_dropout = 0.1
    learning_rate = 1E-3
    batch_size = 32
    num_epochs = 100
    if L==6:
        num_shots = 10000
    elif L==10:
        num_shots = 160000
    elif L==14:
        num_shots = 640000
    accuracy_arr = []
    accuracy_arr_err = []
    gamma_arr_new = []
    itera=4
    for i in range(len(gamma_arr_exp)):
        try:
            arr = np.load("RNN_data/data_hardware/succ_proba_array_weak_hardware_post_LSTM_decoder_{}_learning_rate_{}_num_epochs_{}_dropout_{}_RNN_dropout_{}_batch_size_{}_L_{}_p_{}_num_shots_{}_iter_{}.npy".format(num_RNN_units,learning_rate,num_epochs,dropout,RNN_dropout,batch_size,L,gamma_arr_exp[i],num_shots,itera))
            #accuracy_arr.append(arr[len(arr)-1])
            
            accuracy_arr.append(np.mean(arr))
            accuracy_arr_err.append(np.std(arr)/np.sqrt(2*num_shots))
            gamma_arr_new.append(gamma_arr_exp[i])
        except:
            print("ignored", gamma_arr_exp[i],L)
    return gamma_arr_new,accuracy_arr, accuracy_arr_err




def get_RNN_simulation_data_suc_prob(L,gamma_arr_exp):
    if L==6 :num_RNN_units = 128
    else: num_RNN_units = 128
    dropout = 0.1
    RNN_dropout = 0.1
    learning_rate = 0.001
    batch_size = 32
    num_epochs = 100
    itera=1
    if L==6:
        num_shots = 10000
    elif L==10:
        num_shots = 160000
    elif L==14:
        num_shots = 640000
    accuracy_arr = []
    accuracy_arr_err = []
    gamma_arr_new = []
    for i in range(len(gamma_arr_exp)):
        try:
            arr = np.load("RNN_data/data_weak/succ_proba_array_weak_LSTM_decoder_{}_learning_rate_{}_num_epochs_{}_dropout_{}_RNN_dropout_{}_batch_size_{}_L_{}_p_{}_num_shots_{}_iter_{}.npy".format(num_RNN_units,learning_rate,num_epochs,dropout,RNN_dropout,batch_size,L,gamma_arr_exp[i],num_shots,itera))
            #accuracy_arr.append(arr[len(arr)-1])
            accuracy_arr.append(np.mean(arr))
            # accuracy_arr_err.append(np.std(arr)/np.sqrt(2*num_shots))
            accuracy_arr_err.append(np.std(arr))
            gamma_arr_new.append(gamma_arr_exp[i])
        except:
            print("ignored", gamma_arr_exp[i],L)
    return gamma_arr_new,accuracy_arr,accuracy_arr_err



In [9]:
def get_RNN_hardware_data_acc(L,gamma_arr_exp):
    if L==6 :num_RNN_units = 32
    else: num_RNN_units = 128
    dropout = 0.1
    RNN_dropout = 0.1
    learning_rate = 1E-3
    batch_size = 32
    num_epochs = 100
    if L==6:
        num_shots = 10000
    elif L==10:
        num_shots = 160000
    elif L==14:
        num_shots = 640000
    accuracy_arr = []
    accuracy_arr_err = []
    gamma_arr_new = []
    itera=4
    for i in range(len(gamma_arr_exp)):
        try:
            arr = np.load("RNN_data/data_hardware/accuracy_array_weak_hardware_post_LSTM_decoder_{}_learning_rate_{}_num_epochs_{}_dropout_{}_RNN_dropout_{}_batch_size_{}_L_{}_p_{}_num_shots_{}_iter_{}.npy".format(num_RNN_units,learning_rate,num_epochs,dropout,RNN_dropout,batch_size,L,gamma_arr_exp[i],num_shots,itera))
            #accuracy_arr.append(arr[len(arr)-1])
            
            accuracy_arr.append(np.mean(arr))
            accuracy_arr_err.append(np.std(arr)/np.sqrt(2*num_shots))
            gamma_arr_new.append(gamma_arr_exp[i])
        except:
            print("ignored", gamma_arr_exp[i],L)
    return gamma_arr_new,accuracy_arr, accuracy_arr_err




def get_RNN_simulation_data_acc(L,gamma_arr_exp):
    if L==6 :num_RNN_units = 128
    else: num_RNN_units = 128
    dropout = 0.1
    RNN_dropout = 0.1
    learning_rate = 0.001
    batch_size = 32
    num_epochs = 100
    itera=1
    if L==6:
        num_shots = 10000
    elif L==10:
        num_shots = 160000
    elif L==14:
        num_shots = 640000
    accuracy_arr = []
    accuracy_arr_err = []
    gamma_arr_new = []
    for i in range(len(gamma_arr_exp)):
        try:
            arr = np.load("RNN_data/data_weak/accuracy_array_weak_LSTM_decoder_{}_learning_rate_{}_num_epochs_{}_dropout_{}_RNN_dropout_{}_batch_size_{}_L_{}_p_{}_num_shots_{}_iter_{}.npy".format(num_RNN_units,learning_rate,num_epochs,dropout,RNN_dropout,batch_size,L,gamma_arr_exp[i],num_shots,itera))
            #accuracy_arr.append(arr[len(arr)-1])
            accuracy_arr.append(np.mean(arr))
            accuracy_arr_err.append(np.std(arr)/np.sqrt(2*num_shots))
            gamma_arr_new.append(gamma_arr_exp[i])
        except:
            print("ignored", gamma_arr_exp[i],L)
    return gamma_arr_new,accuracy_arr,accuracy_arr_err


In [17]:
def get_p(name:str):
    p = 0
    index1 = name.index('p=')
    index2 = name.index('_seed=')
    p = float(name[index1+2:index2])
    return p

def get_Q(name:str):
    g = 0
    index1 = name.index('Q=')
    index2 = name.index('_p=')

    g = float(name[index1+2:index2])
    return g

def get_T(name:str):
    T = 0
    index2 = name.index('_Q')
    index1 = name.index('depth=')
    T = int(name[index1+6:index2])
    return T

def get_L(name:str):
    N = 0
    index1 = name.index('L=')
    index2 = name.index('_depth=')
    N = int(name[index1+2:index2])
    return N

def total_samples():

    sample_dict = {}

    root_direc = '../Quantinum/Weak measurements/data/hardware_data/measurement_data_all_qubits_noisy_depth_ratio=0.5'
    for file in os.listdir(root_direc):
        L,Q,p = get_L(file),get_Q(file),get_p(file)
        with open(root_direc+'/'+file,'rb') as f:
            data = pickle.load(f)
        count = 0
        for x in data:
            count += x[1]
        sample_dict[(L,Q,p)] = count
    
    return sample_dict

## Sampling cose plot

In [18]:
# Sampling cost plot
L_list = [6,10,14]
pl.figure(1,figsize=(6,5))

samples = total_samples()
hardware_data_file =  '../Quantinum/Weak measurements/data/hardware_data/biased_sep_data_neel_depth_ratio=0.5/seed=1_all_qubits_special_scrambled_decoding_protocol=3_noisy'
with open(hardware_data_file,'rb') as f:
    hardware_data = pickle.load(f)

cost_data = {}
p_list = [0.079,0.236,0.516,0.785,0.92,1.122]
for p in p_list:
    cost_data[p] = []
    for L in L_list:
        samples_ran = 0
        samples_used = 0
        for Q in [L//2-1,L//2]:
            if (L,Q,p) not in samples:
                continue
            samples_ran += samples[(L,Q,p)]
            samples_used += len(hardware_data[L][p][Q])
        if samples_ran!=0: cost_data[p].append(1-samples_used/samples_ran)
        else: cost_data[p].append(0)

[pl.plot(L_list,cost_data[p],'-o',label=r'$\gamma={}$'.format(round(p*2/np.pi,4))) for p in [0.236,0.785,1.122]]
print(cost_data)
pl.legend(fontsize=16)
pl.ylabel(r'Sampling cost',fontsize=22)
pl.xlabel(r'L',fontsize=22)
pl.xticks([6,10,14],fontsize=22)
pl.yticks(pl.yticks()[0][::2],fontsize=22)
pl.tight_layout()
# pl.plot(p_list,[cost_data[p][2] for p in p_list],'-o')

{0.079: [0.11499999999999999, 0.345, 0], 0.236: [0.09250000000000003, 0.365, 0.57125], 0.516: [0.07499999999999996, 0.38249999999999995, 0.6174999999999999], 0.785: [0.08499999999999996, 0.345, 0.6591666666666667], 0.92: [0.18500000000000005, 0.41000000000000003, 0.654], 1.122: [0.14500000000000002, 0.38749999999999996, 0.665]}


In [19]:
# no. of samples
samples

{(10, 5.0, 0.236): 200,
 (14, 6.0, 0.785): 600,
 (6, 3.0, 0.079): 200,
 (14, 6.0, 1.122): 200,
 (10, 5.0, 1.122): 200,
 (10, 4.0, 0.92): 700,
 (6, 3.0, 0.516): 200,
 (10, 5.0, 0.92): 700,
 (14, 6.0, 0.92): 800,
 (14, 6.0, 0.236): 400,
 (10, 5.0, 0.785): 300,
 (14, 7.0, 0.92): 700,
 (14, 7.0, 1.122): 200,
 (14, 7.0, 0.785): 600,
 (6, 2.0, 0.92): 100,
 (10, 4.0, 0.236): 200,
 (6, 2.0, 0.079): 200,
 (6, 3.0, 0.92): 100,
 (10, 4.0, 0.785): 300,
 (14, 7.0, 0.236): 400,
 (6, 2.0, 0.516): 200,
 (10, 4.0, 1.122): 200,
 (6, 2.0, 0.785): 200,
 (10, 4.0, 0.516): 200,
 (6, 2.0, 1.122): 200,
 (14, 7.0, 0.516): 400,
 (10, 4.0, 0.079): 100,
 (6, 2.0, 0.236): 200,
 (6, 3.0, 1.122): 200,
 (10, 5.0, 0.516): 200,
 (6, 3.0, 0.785): 200,
 (10, 5.0, 0.079): 100,
 (6, 3.0, 0.236): 200,
 (14, 6.0, 0.516): 400}

In [20]:
# values of gamma
np.array(p_list)*2/np.pi

array([0.05029296, 0.15024227, 0.3284958 , 0.49974652, 0.58569019,
       0.71428738])

## Comparing mitigation with no mitigation

In [21]:
# Plot comparing mitigation
L_list = [6,8,10,12,14,16][:5]
L_list = [6,10,14]
fig,ax = pl.subplots(1,1)

colors = pl.cm.Blues(np.linspace(0.5,1,len(L_list)))

depth_ratio=0.5
if depth_ratio != 1:
    depth_label = '_depth_ratio='+str(depth_ratio) + '/'
else:
    depth_label = '/'

ROOT_DIREC_Q = '../Quantinum/Weak measurements/data/hardware_data/'
p_select_Q = [0.079,0.236,0.516,0.785,0.92,1.122] # p values for hardware data
file2_Q = ROOT_DIREC_Q+'biased_sep_data'+depth_label+'seed=1_all_qubits_special_scrambled'
is_noisy = '_noisy'


data2_Q_no_mit = load_data(file2_Q+'_decoding_protocol=0'+is_noisy)
suc_no,err_no,_,_ = get_suc_prob_data(data2_Q_no_mit,L_list,temp_quan=0,p_selected=p_select_Q)

data2_Q = load_data(file2_Q+'_decoding_protocol=3'+is_noisy)
suc,err,_,_ = get_suc_prob_data(data2_Q,L_list,temp_quan=0,p_selected=p_select_Q)

lw=2.5
for i,L in enumerate(L_list):
    xdata = np.array(sorted(list(suc[L].keys())))
    ydata = [suc[L][p] for p in xdata]
    yerr = [err[L][p] for p in xdata]

    ax.errorbar(xdata*2/np.pi,ydata,yerr=yerr,ls='-',color=colors[i],marker='o',lw=2)

    xdata = np.array(sorted(list(suc_no[L].keys())))
    ydata = [suc_no[L][p] for p in xdata]
    yerr = [err_no[L][p] for p in xdata]

    ax.errorbar(xdata*2/np.pi,ydata,yerr=yerr,ls='--',color=colors[i],marker='o',lw=2)

ax.plot([],[],'-',color='k',label='With error mitigation')
ax.plot([],[],':',color='k',label='Without error mitigation')


[ax.plot([],[],'-',label=r'$L=$'+str(L)) for L in L_list]

ax.tick_params(axis='both', which='major', labelsize=14)
ax.tick_params(axis='both', which='major', labelsize=14)

label_size = 16

ax.set_ylabel(r'Credence $C$',fontsize=label_size)


ax.set_xlabel(r'$\gamma$',fontsize=label_size)

ax.legend(fontsize=12)

fig.set_tight_layout(True)

L= 6 p= 0.079 data_size: 200 200
L= 6 p= 0.236 data_size: 200 200
L= 6 p= 0.516 data_size: 200 200
L= 6 p= 0.785 data_size: 200 200
L= 6 p= 0.92 data_size: 97 100
L= 6 p= 1.122 data_size: 196 197
L= 10 p= 0.079 data_size: 100 100
L= 10 p= 0.236 data_size: 200 200
L= 10 p= 0.516 data_size: 199 200
L= 10 p= 0.785 data_size: 285 298
L= 10 p= 0.92 data_size: 664 690
L= 10 p= 1.122 data_size: 190 200
L= 14 p= 0.236 data_size: 400 400
L= 14 p= 0.516 data_size: 399 400
L= 14 p= 0.785 data_size: 566 580
L= 14 p= 0.92 data_size: 636 764
L= 14 p= 1.122 data_size: 180 189
L= 6 p= 0.079 data_size: 181 173
L= 6 p= 0.236 data_size: 183 180
L= 6 p= 0.516 data_size: 188 182
L= 6 p= 0.785 data_size: 187 179
L= 6 p= 0.92 data_size: 84 79
L= 6 p= 1.122 data_size: 174 168
L= 10 p= 0.079 data_size: 64 67
L= 10 p= 0.236 data_size: 136 118
L= 10 p= 0.516 data_size: 128 119
L= 10 p= 0.785 data_size: 206 187
L= 10 p= 0.92 data_size: 442 384
L= 10 p= 1.122 data_size: 125 120
L= 14 p= 0.236 data_size: 164 179
L=

## Comaparing decoders for accuracy

In [22]:
# %matplotlib qt
fig,ax = pl.subplots(1,figsize=(6,5))
L_list = [6,8,10,12,14,16][:5]
# L_list = [6,10,14]

colors = pl.cm.Blues(np.linspace(0.5,1,len(L_list)))


depth_ratio=0.5
if depth_ratio != 1:
    depth_label = '_depth_ratio='+str(depth_ratio) + '/'
else:
    depth_label = '/'


quantinuum = True
ROOT_DIREC_Q = '../Quantinum/Weak measurements/data/hardware_data/'
p_select_Q = [0.079,0.236,0.516,0.785,0.92,1.122]
    
ROOT_DIREC = 'data/' #simulation data directory

p_list = list(np.round(np.linspace(0.2*np.pi/2,0.8*np.pi/2,15),3))
p_list.extend(list(np.array([0.001,0.01,0.02,0.05,0.1,0.15])*np.pi/2))
p_list = sorted(p_list)

# p_select is a list of p values we want to plot
p_select = np.array([0.07853981633974483,0.23561944901923448 ,0.516,0.785,0.92,1.122]) # p values ran on hardware
p_select = [p_list[0]] + p_list[3:]

is_noisy = '_noisy'
decoding_label = '_decoding_protocol='+str(3)

#_large label is for simulation data with 10000 samples

file_large = './large_scale_data/sep_seed=1_all_qubits_special_scrambled'

file2_Q = ROOT_DIREC_Q+'biased_sep_data_neel'+depth_label+'seed=1_all_qubits_special_scrambled'+decoding_label+is_noisy
file2_large = 'large_scale_data/biased_sep_seed=1_all_qubits_special_scrambled'

file3_Q = ROOT_DIREC_Q+'quantum_decoder_neel'+depth_label+'/seed=1_all_qubits_special_scrambled'+decoding_label+is_noisy
file3_large = 'large_scale_data/quantum_decoder_seed=1_all_qubits_special_scrambled'

data = load_data(file_large)
data2 = load_data(file2_large)
data3 = load_data(file3_large)

#Hardware date
data2_Q = load_data(file2_Q)
data3_Q = load_data(file3_Q)

data_list = [data2,data3]
temp_quan = [0,0]

plot_quantum_data = False

plot_suc = True

if plot_quantum_data:
    p_select = p_select_Q
    data_list = [data2_Q,data3_Q]
    temp_quan = [0,1] #temp_quan = 1 for quantum decoder data on hardware. The quantum data has a different data structure

lss = ['-',':']
lw = 2
for i,dat in enumerate(data_list):
    if not plot_suc:
        acc,err,_,_ = get_acc_data(dat,L_list,temp_quan=temp_quan[i],p_selected=p_select) #use get_suc_prob_data for suc_prob
    else:
        acc,err,_,_ = get_suc_prob_data(dat,L_list,temp_quan=temp_quan[i],p_selected=p_select)
    for j,L in enumerate(L_list):
        xdata = np.array(sorted(list(acc[L].keys())))
        ydata = np.array([acc[L][p] for p in xdata])
        errdata = np.array([err[L][p] for p in xdata])

        ax.errorbar(xdata*2/np.pi,ydata,yerr=errdata,ls=lss[i],marker='o',ms=6,color=colors[j],lw=lw)


for i,L in enumerate(L_list):
        # use get_RNN_hardware_data_suc_prob for suc_prob

        if plot_quantum_data:
            if plot_suc:
                xdata,ydata,yerr = get_RNN_hardware_data_suc_prob(L,p_select)
            else:
                xdata,ydata,yerr = get_RNN_hardware_data_acc(L,p_select)
        else:
            if plot_suc:
                xdata,ydata,yerr = get_RNN_simulation_data_suc_prob(L,sorted(p_select))
            else:
                xdata,ydata,yerr = get_RNN_simulation_data_acc(L,sorted(p_select))
        ax.errorbar([p*2/np.pi for p in xdata],ydata,yerr=yerr,ls='--',marker='o',color=colors[i],lw=lw)
        continue

# RNN_plot(L_list,p_select_Q)

pl.plot([],[],'-',color='k',label='Stat-mech')
pl.plot([],[],'--',color='k',label='RNN')
pl.plot([],[],':',color='k',label='PostBQP')

[ax.plot([],[],'-',label=r'$L=$'+str(L)) for L in L_list]

ax.tick_params(axis='both', which='major', labelsize=14)
ax.tick_params(axis='both', which='major', labelsize=14)

label_size = 16

if plot_suc:
    ax.set_ylabel(r'Credence $C$',fontsize=label_size)
else:
    ax.set_ylabel(r'Accuracy $\alpha$',fontsize=label_size)


ax.set_xlabel(r'$\gamma$',fontsize=label_size)


ax.legend(fontsize=12)

fig.set_tight_layout(True)

L= 6 p= 0.07853981633974483 data_size: 10000 10000
L= 6 p= 0.15707963267948966 data_size: 10000 10000
L= 6 p= 0.23561944901923448 data_size: 10000 10000
L= 6 p= 0.314 data_size: 10000 10000
L= 6 p= 0.381 data_size: 10000 10000
L= 6 p= 0.449 data_size: 10000 10000
L= 6 p= 0.516 data_size: 10000 10000
L= 6 p= 0.583 data_size: 10000 10000
L= 6 p= 0.651 data_size: 10000 10000
L= 6 p= 0.718 data_size: 10000 10000
L= 6 p= 0.785 data_size: 10000 10000
L= 6 p= 0.853 data_size: 10000 10000
L= 6 p= 0.92 data_size: 10000 10000
L= 6 p= 0.987 data_size: 10000 10000
L= 6 p= 1.055 data_size: 10000 10000
L= 6 p= 1.122 data_size: 10000 10000
L= 6 p= 1.189 data_size: 10000 10000
L= 6 p= 1.257 data_size: 10000 10000
L= 8 p= 0.07853981633974483 data_size: 10000 10000
L= 8 p= 0.15707963267948966 data_size: 10000 10000
L= 8 p= 0.23561944901923448 data_size: 10000 10000
L= 8 p= 0.314 data_size: 10000 10000
L= 8 p= 0.381 data_size: 10000 10000
L= 8 p= 0.449 data_size: 10000 10000
L= 8 p= 0.516 data_size: 1000

NameError: name 'get_RNN_simulation_data_suc_prob' is not defined

### Main Figure

In [23]:
L_list = [6,8,10,12,14,16][:5]
L_list = [6,10,14]

depth_ratio=0.5
if depth_ratio != 1:
    depth_label = '_depth_ratio='+str(depth_ratio) + '/'
else:
    depth_label = '/'

fig,axs = pl.subplots(1,2,figsize=(11,5))

quantinuum = True
ROOT_DIREC_Q = '../Quantinum/Weak measurements/data/hardware_data/'
p_select_Q = [0.079,0.236,0.516,0.785,0.92,1.122]
    
ROOT_DIREC = 'data/'
p_select = np.array([0.07853981633974483,0.23561944901923448 ,0.516,0.785,0.92,1.122])
is_noisy = '_noisy'
decoding_label_3 = '_decoding_protocol='+str(3)
decoding_label_0 = '_decoding_protocol='+str(0)

file_large = 'large_scale_data/biased_sep_seed=1_all_qubits_special_scrambled'
file = ROOT_DIREC+'sep_data_neel'+depth_label+'seed=1_all_qubits_special_scrambled'+decoding_label_3+is_noisy

file2 = ROOT_DIREC+'biased_sep_data_neel'+depth_label+'seed=1_all_qubits_special_scrambled'+decoding_label_3+is_noisy
file2_Q = ROOT_DIREC_Q+'biased_sep_data_neel'+depth_label+'seed=1_all_qubits_special_scrambled'+decoding_label_3+is_noisy

file3 = ROOT_DIREC + 'quantum_decoder_neel'+depth_label+'/seed=1_all_qubits_special_scrambled'
file3_Q = ROOT_DIREC_Q+'quantum_decoder_neel'+depth_label+'/seed=1_all_qubits_special_scrambled'+decoding_label_3+is_noisy
      
data2_Q = load_data(file2_Q)
data2 = load_data(file_large)


suc_prob_sim, err_sim, var_sim, var_err_sim = get_suc_prob_data(data2,L_list,temp_quan=0,p_selected=[])
suc_prob_exp, err_exp, var_exp, var_err_exp = get_suc_prob_data(data2_Q,L_list,temp_quan=0,p_selected=[])



lw = 3 
for i,L in enumerate(L_list):
    p_data_sim = np.array(sorted(list(suc_prob_sim[L].keys())))
    ydata_sim = [suc_prob_sim[L][p] for p in p_data_sim]
    errdata_sim = [err_sim[L][p] for p in p_data_sim]
    var_ydata_sim = [var_sim[L][p] for p in p_data_sim]
    var_errdata_sim = [var_err_sim[L][p] for p in p_data_sim]

    p_data_exp = np.array(sorted(list(suc_prob_exp[L].keys())))
    ydata_exp = [suc_prob_exp[L][p] for p in p_data_exp]
    errdata_exp = [err_exp[L][p] for p in p_data_exp]
    var_ydata_exp = [var_exp[L][p] for p in p_data_exp]
    var_errdata_exp = [var_err_exp[L][p] for p in p_data_exp]

    axs[0].errorbar(p_data_sim/(np.pi/2),ydata_sim,yerr=errdata_sim,marker='o',ls=':',color=colors[i],ms=3,alpha=0.6,lw=2)

    axs[0].errorbar(p_data_exp/(np.pi/2),ydata_exp,yerr=errdata_exp,marker='o',ls='-',color=colors[i],ms=6,alpha=1,lw=lw)

    axs[1].errorbar(p_data_sim/(np.pi/2),var_ydata_sim,yerr=var_errdata_sim,marker='o',ls=':',color=colors[i],ms=3,alpha=0.4)

    axs[1].errorbar(p_data_exp/(np.pi/2),var_ydata_exp,yerr=var_errdata_exp,marker='o',ls='-',color=colors[i],ms=6,alpha=1,lw=lw)



axs[0].errorbar([],[],[],ls='-',marker='o',ms=6,color='k',label='Experiment')
axs[0].errorbar([],[],[],ls=':',marker='o',ms=3,color='k',label='Noiseless Simulation')
# [pl.plot([],[],'-',label=r'$L=$'+str(L)) for L in L_list]

axs[0].tick_params(axis='both', which='major', labelsize=14)
axs[1].tick_params(axis='both', which='major', labelsize=14)

label_size = 16

axs[0].set_ylabel(r'Credence $C$',fontsize=label_size)
axs[1].set_ylabel(r'Variance of $C$',fontsize=label_size)

axs[0].set_xlabel(r'$\gamma$',fontsize=label_size)
axs[1].set_xlabel(r'$\gamma$',fontsize=label_size)

axs[0].legend(fontsize=12)

fig.set_tight_layout(True)

L= 6 p= 0.07853981633974483 data_size: 10000 10000
L= 6 p= 0.15707963267948966 data_size: 10000 10000
L= 6 p= 0.23561944901923448 data_size: 10000 10000
L= 6 p= 0.314 data_size: 10000 10000
L= 6 p= 0.381 data_size: 10000 10000
L= 6 p= 0.449 data_size: 10000 10000
L= 6 p= 0.516 data_size: 10000 10000
L= 6 p= 0.583 data_size: 10000 10000
L= 6 p= 0.651 data_size: 10000 10000
L= 6 p= 0.718 data_size: 10000 10000
L= 6 p= 0.785 data_size: 10000 10000
L= 6 p= 0.853 data_size: 10000 10000
L= 6 p= 0.92 data_size: 10000 10000
L= 6 p= 0.987 data_size: 10000 10000
L= 6 p= 1.055 data_size: 10000 10000
L= 6 p= 1.122 data_size: 10000 10000
L= 6 p= 1.189 data_size: 10000 10000
L= 6 p= 1.257 data_size: 10000 10000
L= 10 p= 0.07853981633974483 data_size: 10000 10000
L= 10 p= 0.15707963267948966 data_size: 10000 10000
L= 10 p= 0.23561944901923448 data_size: 10000 10000
L= 10 p= 0.314 data_size: 10000 10000
L= 10 p= 0.381 data_size: 10000 10000
L= 10 p= 0.449 data_size: 10000 10000
L= 10 p= 0.516 data_siz

## Sim data

In [24]:
L_list = [6,8,10,12,14,16][:5]

colors = pl.cm.Blues(np.linspace(0.5,1,len(L_list)))

depth_ratio=0.5
if depth_ratio != 1:
    depth_label = '_depth_ratio='+str(depth_ratio) + '/'
else:
    depth_label = '/'

fig,axs = pl.subplots(1,2,figsize=(11,5))


    
ROOT_DIREC = 'data/'
# p_select = np.array([0.07853981633974483,0.23561944901923448 ,0.516,0.785,0.92,1.122])
is_noisy = ''
# decoding_label_3 = '_decoding_protocol='+str(3)
decoding_label_0 = '_decoding_protocol='+str(0)

file_large = 'large_scale_data/biased_sep_seed=1_all_qubits_special_scrambled'

# file_large = 'large_scale_data/quantum_decoder_seed=1_all_qubits_special_scrambled'


      



RNN = False
lw = 3
ms=6 
if not RNN:
    data2 = load_data(file_large)

    suc_prob_sim, err_sim, var_sim, var_err_sim = get_suc_prob_data(data2,L_list,temp_quan=0,p_selected=[])
    for i,L in enumerate(L_list):
        p_data_sim = np.array(sorted(list(suc_prob_sim[L].keys())))
        ydata_sim = [suc_prob_sim[L][p] for p in p_data_sim]
        errdata_sim = [err_sim[L][p] for p in p_data_sim]
        var_ydata_sim = [var_sim[L][p] for p in p_data_sim]
        var_errdata_sim = [var_err_sim[L][p] for p in p_data_sim]


        axs[0].errorbar(p_data_sim/(np.pi/2),ydata_sim,yerr=errdata_sim,marker='o',ls='-',color=colors[i],ms=ms,alpha=1,lw=lw,label=r'$L={}$'.format(L))


        axs[1].errorbar(p_data_sim/(np.pi/2),var_ydata_sim,yerr=var_errdata_sim,marker='o',ls='-',color=colors[i],ms=ms,alpha=1,lw=lw)


if RNN:
    for i,L in enumerate(L_list):
        p_data_sim,ydata_sim,var_ydata_sim = get_RNN_simulation_data_suc_prob(L,sorted(p_select))
        p_data_sim = np.array(p_data_sim)
        axs[0].plot(p_data_sim/(np.pi/2),ydata_sim,marker='o',ls='-',color=colors[i],ms=ms,alpha=1,lw=lw,label=r'$L={}$'.format(L))


        axs[1].plot(p_data_sim/(np.pi/2),var_ydata_sim,marker='o',ls='-',color=colors[i],ms=ms,alpha=1,lw=lw)

# [pl.plot([],[],'-',label=r'$L=$'+str(L)) for L in L_list]

axs[0].tick_params(axis='both', which='major', labelsize=14)
axs[1].tick_params(axis='both', which='major', labelsize=14)

label_size = 16

axs[0].set_ylabel(r'Credence $C$',fontsize=label_size)
axs[1].set_ylabel(r'Variance of $C$',fontsize=label_size)

axs[0].set_xlabel(r'$\gamma$',fontsize=label_size)
axs[1].set_xlabel(r'$\gamma$',fontsize=label_size)

axs[0].legend(fontsize=12)

fig.set_tight_layout(True)

L= 6 p= 0.07853981633974483 data_size: 10000 10000
L= 6 p= 0.15707963267948966 data_size: 10000 10000
L= 6 p= 0.23561944901923448 data_size: 10000 10000
L= 6 p= 0.314 data_size: 10000 10000
L= 6 p= 0.381 data_size: 10000 10000
L= 6 p= 0.449 data_size: 10000 10000
L= 6 p= 0.516 data_size: 10000 10000
L= 6 p= 0.583 data_size: 10000 10000
L= 6 p= 0.651 data_size: 10000 10000
L= 6 p= 0.718 data_size: 10000 10000
L= 6 p= 0.785 data_size: 10000 10000
L= 6 p= 0.853 data_size: 10000 10000
L= 6 p= 0.92 data_size: 10000 10000
L= 6 p= 0.987 data_size: 10000 10000
L= 6 p= 1.055 data_size: 10000 10000
L= 6 p= 1.122 data_size: 10000 10000
L= 6 p= 1.189 data_size: 10000 10000
L= 6 p= 1.257 data_size: 10000 10000
L= 8 p= 0.07853981633974483 data_size: 10000 10000
L= 8 p= 0.15707963267948966 data_size: 10000 10000
L= 8 p= 0.23561944901923448 data_size: 10000 10000
L= 8 p= 0.314 data_size: 10000 10000
L= 8 p= 0.381 data_size: 10000 10000
L= 8 p= 0.449 data_size: 10000 10000
L= 8 p= 0.516 data_size: 1000

## Ancilla data

In [32]:
L_list = [6,8,10,12,14,16][:5]

colors = pl.cm.Blues(np.linspace(0.5,1,len(L_list)))

depth_ratio=0.5
if depth_ratio != 1:
    depth_label = '_depth_ratio='+str(depth_ratio) + '/'
else:
    depth_label = '/'

fig,axs = pl.subplots(1,2,figsize=(11,5))


    
ROOT_DIREC = 'data/'
# p_select = np.array([0.07853981633974483,0.23561944901923448 ,0.516,0.785,0.92,1.122])
is_noisy = ''
# decoding_label_3 = '_decoding_protocol='+str(3)
decoding_label_0 = '_decoding_protocol='+str(0)

file_large = 'large_scale_data/biased_sep_seed=1_all_qubits_special_scrambled'

file_large = 'large_scale_data/quantum_decoder_seed=1_all_qubits_special_scrambled'


      



RNN = False
lw = 3
ms=6 
if not RNN:
    data2 = load_data(file_large)

    suc_prob_sim, err_sim, var_sim, var_err_sim = get_ancilla_data(data2,L_list,temp_quan=0,p_selected=[])
    for i,L in enumerate(L_list):
        p_data_sim = np.array(sorted(list(suc_prob_sim[L].keys())))
        ydata_sim = np.array([suc_prob_sim[L][p] for p in p_data_sim])
        errdata_sim = [err_sim[L][p] for p in p_data_sim]
        var_ydata_sim = [var_sim[L][p] for p in p_data_sim]
        var_errdata_sim = [var_err_sim[L][p] for p in p_data_sim]


        axs[0].errorbar((p_data_sim/(np.pi/2)-0.457)*L**(3/4),ydata_sim*L**(1),yerr=errdata_sim,marker='o',ls='-',color=colors[i],ms=ms,alpha=1,lw=lw,label=r'$L={}$'.format(L))


        axs[1].errorbar(p_data_sim/(np.pi/2),var_ydata_sim,yerr=var_errdata_sim,marker='o',ls='-',color=colors[i],ms=ms,alpha=1,lw=lw)


if RNN:
    for i,L in enumerate(L_list):
        p_data_sim,ydata_sim,var_ydata_sim = get_RNN_simulation_data_suc_prob(L,sorted(p_select))
        p_data_sim = np.array(p_data_sim)
        axs[0].plot(p_data_sim/(np.pi/2),ydata_sim,marker='o',ls='-',color=colors[i],ms=ms,alpha=1,lw=lw,label=r'$L={}$'.format(L))


        axs[1].plot(p_data_sim/(np.pi/2),var_ydata_sim,marker='o',ls='-',color=colors[i],ms=ms,alpha=1,lw=lw)

# [pl.plot([],[],'-',label=r'$L=$'+str(L)) for L in L_list]

axs[0].tick_params(axis='both', which='major', labelsize=14)
axs[1].tick_params(axis='both', which='major', labelsize=14)

label_size = 16

axs[0].set_ylabel(r'$S_R$',fontsize=label_size+2)
axs[1].set_ylabel(r'Variance of $S_R$',fontsize=label_size)

axs[0].set_xlabel(r'$\gamma$',fontsize=label_size)
axs[1].set_xlabel(r'$\gamma$',fontsize=label_size)

axs[0].legend(fontsize=12)

fig.set_tight_layout(True)

L= 6 p= 0.07853981633974483 data_size: 10000 10000
L= 6 p= 0.15707963267948966 data_size: 10000 10000
L= 6 p= 0.23561944901923448 data_size: 10000 10000
L= 6 p= 0.314 data_size: 10000 10000
L= 6 p= 0.381 data_size: 10000 10000
L= 6 p= 0.449 data_size: 10000 10000
L= 6 p= 0.516 data_size: 10000 10000
L= 6 p= 0.583 data_size: 10000 10000
L= 6 p= 0.651 data_size: 10000 10000
L= 6 p= 0.718 data_size: 10000 10000
L= 6 p= 0.785 data_size: 10000 10000
L= 6 p= 0.853 data_size: 10000 10000
L= 6 p= 0.92 data_size: 10000 10000
L= 6 p= 0.987 data_size: 10000 10000
L= 6 p= 1.055 data_size: 10000 10000
L= 6 p= 1.122 data_size: 10000 10000
L= 6 p= 1.189 data_size: 10000 10000
L= 6 p= 1.257 data_size: 10000 10000
L= 8 p= 0.07853981633974483 data_size: 10000 10000
L= 8 p= 0.15707963267948966 data_size: 10000 10000
L= 8 p= 0.23561944901923448 data_size: 10000 10000
L= 8 p= 0.314 data_size: 10000 10000
L= 8 p= 0.381 data_size: 10000 10000
L= 8 p= 0.449 data_size: 10000 10000
L= 8 p= 0.516 data_size: 1000

## Binder ratio

In [25]:
L_list = [6,8,10,12,14,16][:5]

colors = pl.cm.Blues(np.linspace(0.5,1,len(L_list)))

depth_ratio=0.5
if depth_ratio != 1:
    depth_label = '_depth_ratio='+str(depth_ratio) + '/'
else:
    depth_label = '/'

fig,axs = pl.subplots(1,1,figsize=(6,5))

    
ROOT_DIREC = 'data/'
# p_select = np.array([0.07853981633974483,0.23561944901923448 ,0.516,0.785,0.92,1.122])
is_noisy = ''
# decoding_label_3 = '_decoding_protocol='+str(3)
decoding_label_0 = '_decoding_protocol='+str(0)

file_large = 'large_scale_data/biased_sep_seed=1_all_qubits_special_scrambled'

file_large = 'large_scale_data/quantum_decoder_seed=1_all_qubits_special_scrambled'

lw = 2
ms=6 
if not RNN:
    data2 = load_data(file_large)

    binder = get_binder_data(data2,L_list,temp_quan=0,p_selected=[])
    for i,L in enumerate(L_list):
        p_data_sim = np.array(sorted(list(suc_prob_sim[L].keys())))
        ydata_sim = [binder[L][p] for p in p_data_sim]
        
        axs.plot(p_data_sim/(np.pi/2),ydata_sim,marker='o',ls='-',color=colors[i],ms=ms,alpha=1,lw=lw,label=r'$L={}$'.format(L))


axs.tick_params(axis='both', which='major', labelsize=14)

label_size = 16

axs.set_ylabel(r'Binder ratio',fontsize=label_size+2)

axs.set_xlabel(r'$\gamma$',fontsize=label_size)

axs.legend(fontsize=12)

# axs.set_ylim(-10,0.5)
fig.set_tight_layout(True)

L= 6 p= 0.07853981633974483 data_size: 10000 10000
L= 6 p= 0.15707963267948966 data_size: 10000 10000
L= 6 p= 0.23561944901923448 data_size: 10000 10000
L= 6 p= 0.314 data_size: 10000 10000
L= 6 p= 0.381 data_size: 10000 10000
L= 6 p= 0.449 data_size: 10000 10000
L= 6 p= 0.516 data_size: 10000 10000
L= 6 p= 0.583 data_size: 10000 10000
L= 6 p= 0.651 data_size: 10000 10000
L= 6 p= 0.718 data_size: 10000 10000
L= 6 p= 0.785 data_size: 10000 10000
L= 6 p= 0.853 data_size: 10000 10000
L= 6 p= 0.92 data_size: 10000 10000
L= 6 p= 0.987 data_size: 10000 10000
L= 6 p= 1.055 data_size: 10000 10000
L= 6 p= 1.122 data_size: 10000 10000
L= 6 p= 1.189 data_size: 10000 10000
L= 6 p= 1.257 data_size: 10000 10000
L= 8 p= 0.07853981633974483 data_size: 10000 10000
L= 8 p= 0.15707963267948966 data_size: 10000 10000
L= 8 p= 0.23561944901923448 data_size: 10000 10000
L= 8 p= 0.314 data_size: 10000 10000
L= 8 p= 0.381 data_size: 10000 10000
L= 8 p= 0.449 data_size: 10000 10000
L= 8 p= 0.516 data_size: 1000

In [117]:
os.listdir('../Quantinum/Weak measurements/emulator_data/')

FileNotFoundError: [Errno 2] No such file or directory: '../Quantinum/Weak measurements/emulator_data/'

### Figure for proposal

In [1]:
# %matplotlib qt
pl.figure(1,figsize=(16,5))
L_list = [6,8,10,12,16][:4][:4]
pl.suptitle('Qiskit data',fontsize=20)
file = 'data_quantum_decoder/decoder/seed=1_all_qubits_special_scrambled_decoding_protocol='

file2 = 'data/sep_data/seed=1_all_qubits_special_scrambled_decoding_protocol='
prop_cycle = pl.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color'][:]

protocols = [-1,0,3]



for i,decoding_protocol in enumerate(protocols):
    pl.subplot(1,3,i+1)
    if i==0:
        pl.ylabel(r'Success prob',fontsize=16)

    if decoding_protocol == -1:
        is_noisy = ''
        decoding_protocol = 0
        noisy_label = 'No noise'
    else:
        is_noisy = '_noisy'
        noisy_label = 'Noisy'
    
    if decoding_protocol == 0:
        decoding_label = 'No mitigation'
    elif decoding_protocol == 3:
        decoding_label = 'Error mitigated'
    # data=load_data(file+str(decoding_protocol)+is_noisy)
    data2 = load_data(file2+str(decoding_protocol)+is_noisy)
    print(data.keys())
    N_samples = 300
    plot_success_prob(data,L_list,charge_fac=0,marker='s',N_samples=N_samples,color_list=colors)
    plot_success_prob(data2,L_list,charge_fac=0,marker='s',ls='--',N_samples=N_samples,color_list=colors,temp_quan=0,post_label='_SEP')

    # plot_success_binder(data,L_list,marker='s',N_samples=-1)
    pl.title(noisy_label+'. '+ decoding_label + '. ' + 'shots = {}'.format(N_samples),fontsize=16)
    pl.tight_layout()
    pl.ylim(0.45,1.05)


NameError: name 'pl' is not defined

In [130]:
%matplotlib qt
pl.figure(1)

data=load_data('data/biased_sep_data/seed=1_all_qubits_special_scrambled')
L_list = [6,8,10,12,14,16][::2]
plot_success_ent(data,L_list,charge_fac=0,marker='s')
# plot_success_binder(data,L_list,marker='s',N_samples=-1)
pl.title('Biased decoder. Not Noisy, N_samples = 600',fontsize=16)
pl.tight_layout()
pl.xlim(0.2,0.5)
# pl.ylim(-2,1)


pl.figure(2)
# pl.ylim(0,np.log(2))
data=load_data('data/biased_sep_data/seed=1_all_qubits_special_scrambled')
L_list = [6,8,10,12,14,16][:3]
plot_success_ent(data,L_list,charge_fac=0,marker='s')
# plot_success_binder(data,L_list,marker='s',N_samples=-1)
pl.title('Biased decoder. Noisy, N_samples = 600',fontsize=16)
pl.tight_layout()
pl.xlim(0.2,0.5)
# pl.ylim(-2,1)

L= 6 p= 0.412 data_size: 299 299
L= 6 p= 0.518 data_size: 299 299
L= 6 p= 0.624 data_size: 299 299
L= 6 p= 0.73 data_size: 299 299
L= 6 p= 0.836 data_size: 299 299
L= 10 p= 0.412 data_size: 299 299
L= 10 p= 0.518 data_size: 299 299
L= 10 p= 0.624 data_size: 299 299
L= 10 p= 0.73 data_size: 299 299
L= 10 p= 0.836 data_size: 299 299
L= 14 p= 0.412 data_size: 299 299
L= 14 p= 0.518 data_size: 299 299
L= 14 p= 0.624 data_size: 299 299
L= 14 p= 0.73 data_size: 299 299
L= 14 p= 0.836 data_size: 299 299
L= 6 p= 0.412 data_size: 299 299
L= 6 p= 0.518 data_size: 299 299
L= 6 p= 0.624 data_size: 299 299
L= 6 p= 0.73 data_size: 299 299
L= 6 p= 0.836 data_size: 299 299
L= 8 p= 0.412 data_size: 299 299
L= 8 p= 0.518 data_size: 299 299
L= 8 p= 0.624 data_size: 299 299
L= 8 p= 0.73 data_size: 299 299
L= 8 p= 0.836 data_size: 299 299
L= 10 p= 0.412 data_size: 299 299
L= 10 p= 0.518 data_size: 299 299
L= 10 p= 0.624 data_size: 299 299
L= 10 p= 0.73 data_size: 299 299
L= 10 p= 0.836 data_size: 299 299


(0.2, 0.5)

In [25]:
%matplotlib qt
pl.figure(1)

data=load_data('data/sep_data/seed=1_all_qubits_special_scrambled')
L_list = [6,8,10,12,14,16][:1]
plot_success_ent(data,L_list,charge_fac=0,marker='s',N_samples=300)
# pl.ylim(0,np.log(2))
pl.title('Not noisy, N_samples = 300',fontsize=16)
pl.tight_layout()

pl.figure(2)
# pl.ylim(0,np.log(2))
data=load_data('data/sep_data/seed=1_all_qubits_special_scrambled_noisy')
L_list = [6,8,10,12,14,16][:1]
plot_success_ent(data,L_list,charge_fac=0,marker='s',N_samples=45)
pl.title('Noisy, N_samples = 50',fontsize=16)
pl.tight_layout()


L= 6 p= 0.2 data_size: 300 300
L= 6 p= 0.253 data_size: 300 300
L= 6 p= 0.306 data_size: 300 300
L= 6 p= 0.359 data_size: 300 300
L= 6 p= 0.412 data_size: 300 300
L= 6 p= 0.465 data_size: 300 300
L= 6 p= 0.518 data_size: 300 300
L= 6 p= 0.571 data_size: 300 300
L= 6 p= 0.624 data_size: 300 300
L= 6 p= 0.677 data_size: 300 300
L= 6 p= 0.73 data_size: 300 300
L= 6 p= 0.783 data_size: 300 300
L= 6 p= 0.836 data_size: 300 300
L= 6 p= 0.889 data_size: 300 300
L= 6 p= 0.942 data_size: 300 300
L= 6 p= 0.2 data_size: 45 45
L= 6 p= 0.253 data_size: 45 45
L= 6 p= 0.306 data_size: 45 45
L= 6 p= 0.359 data_size: 45 45
L= 6 p= 0.412 data_size: 45 45
L= 6 p= 0.465 data_size: 45 45
L= 6 p= 0.518 data_size: 45 45
L= 6 p= 0.571 data_size: 45 45
L= 6 p= 0.624 data_size: 45 45
L= 6 p= 0.677 data_size: 45 45
L= 6 p= 0.73 data_size: 45 45
L= 6 p= 0.783 data_size: 45 45
L= 6 p= 0.836 data_size: 45 45
L= 6 p= 0.889 data_size: 45 45
L= 6 p= 0.942 data_size: 45 45


In [21]:
# Function to ensure that p_suc for initial states with charge Q has non-zero probabilities
def sanity_check(data,L_list,p_list):
    for L in L_list[:]:
        for p in p_list:
            p_Q = list(data[L][p][L//2])
            p_Q2 = list(data[L][p][L//2-1])
            if np.any(p_Q==0) or np.any(p_Q2==0):
                print("There are some zero probabilities")
            else:
                print("L =",L,"p =",p,"Sanity check complete; no zero probabilities")

In [10]:
sanity_check(data,L_list,p_list)

L = 8 p = 0.05 Sanity check complete; no zero probabilities
L = 8 p = 0.1 Sanity check complete; no zero probabilities
L = 8 p = 0.13 Sanity check complete; no zero probabilities
L = 8 p = 0.16 Sanity check complete; no zero probabilities
L = 8 p = 0.2 Sanity check complete; no zero probabilities
L = 8 p = 0.25 Sanity check complete; no zero probabilities
L = 8 p = 0.3 Sanity check complete; no zero probabilities
L = 10 p = 0.05 Sanity check complete; no zero probabilities
L = 10 p = 0.1 Sanity check complete; no zero probabilities
L = 10 p = 0.13 Sanity check complete; no zero probabilities
L = 10 p = 0.16 Sanity check complete; no zero probabilities
L = 10 p = 0.2 Sanity check complete; no zero probabilities
L = 10 p = 0.25 Sanity check complete; no zero probabilities
L = 10 p = 0.3 Sanity check complete; no zero probabilities
L = 12 p = 0.05 Sanity check complete; no zero probabilities
L = 12 p = 0.1 Sanity check complete; no zero probabilities
L = 12 p = 0.13 Sanity check complete;

In [43]:
aa = np.eye(6)

In [44]:
for i in range(6):
    aa[i,(3+i)%6]=1

In [47]:
aa[[3,4,5],[3,4,5]]=-1

In [56]:
aa=aa/2**0.5

In [57]:
a,b = np.linalg.eig(aa)

In [58]:
b.T/2**0.5

array([[ 0.65328148,  0.        ,  0.        ,  0.27059805,  0.        ,
         0.        ],
       [ 0.27059805,  0.        ,  0.        , -0.65328148,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.65328148,  0.        ,  0.        ,
         0.27059805],
       [ 0.        ,  0.        ,  0.27059805,  0.        ,  0.        ,
        -0.65328148],
       [ 0.        ,  0.27059805,  0.        ,  0.        , -0.65328148,
         0.        ],
       [ 0.        , -0.65328148,  0.        ,  0.        , -0.27059805,
         0.        ]])

In [69]:
a,b.T

(array([ 1., -1.,  1., -1., -1.,  1.]),
 array([[ 0.92387953,  0.        ,  0.        ,  0.38268343,  0.        ,
          0.        ],
        [ 0.38268343,  0.        ,  0.        , -0.92387953,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.92387953,  0.        ,  0.        ,
          0.38268343],
        [ 0.        ,  0.        ,  0.38268343,  0.        ,  0.        ,
         -0.92387953],
        [ 0.        ,  0.38268343,  0.        ,  0.        , -0.92387953,
          0.        ],
        [ 0.        , -0.92387953,  0.        ,  0.        , -0.38268343,
          0.        ]]))

In [64]:
0.92387953**2

0.8535533859530209

In [68]:
1/10**0.5

0.31622776601683794

In [153]:
with open("../../quantum_decoder_seed=1_all_qubits_special_scrambled",'rb') as f:
    data = pickle.load(f)


In [154]:
data.keys()

dict_keys([6, 8, 10, 12])

qt.qpa.backingstore: Back buffer dpr of 1 doesn't match <_NSViewBackingLayer: 0x2d0026da0> contents scale of 2 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 2 doesn't match <_NSViewBackingLayer: 0x2d0026da0> contents scale of 1 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 2 doesn't match <_NSViewBackingLayer: 0x2d0026da0> contents scale of 1 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 1 doesn't match <_NSViewBackingLayer: 0x2d0026da0> contents scale of 2 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 1 doesn't match <_NSViewBackingLayer: 0x2d0026da0> contents scale of 2 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 2 doesn't match <_NSViewBackingLayer: 0x2d0026da0> contents scale of 1 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 2 doesn't match <_NSViewBackingLayer: 0x2d0026da0> contents scale of 1 - updating layer to match.
qt.qpa.backingstore: Back buffer d

In [210]:
with open('data/post_selected_traj_BIASED','rb') as f:
    data = pickle.load(f)

In [213]:
(data[(10, 0.236, 5)])[0]

(array([[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
        [-1, -1, -1, -1,  1, -1,  1,  1,  1,  1]]),
 1)